In [1]:
# Starter code
from pyspark.sql import SparkSession

# Create spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

# Read in full sparkify dataset
event_data = "s3n://udacity-dsnd/sparkify/sparkify_event_data.json"
df = spark.read.json(event_data)
df.head()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1560128379917_0001,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.
Row(artist=u'Popol Vuh', auth=u'Logged In', firstName=u'Shlok', gender=u'M', itemInSession=278, lastName=u'Johnson', length=524.32934, level=u'paid', location=u'Dallas-Fort Worth-Arlington, TX', method=u'PUT', page=u'NextSong', registration=1533734541000, sessionId=22683, song=u'Ich mache einen Spiegel - Dream Part 4', status=200, ts=1538352001000, userAgent=u'"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId=u'1749042')

## Import libraries

In [2]:

# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, concat, desc, explode, lit, min, max, split, udf, count, when, isnull, collect_list
from pyspark.sql.types import IntegerType, BooleanType, FloatType
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.storagelevel import StorageLevel
from pyspark.ml.classification import LogisticRegression, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import datetime
from pyspark.ml import Pipeline 
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, LinearSVC, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics

VBox()

In [3]:
print((df.count(), len(df.columns)))

VBox()

(26259199, 18)

In [4]:
df.columns

VBox()

['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'method', 'page', 'registration', 'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId']

In [5]:
df.select('page').distinct().show(50)

VBox()

+--------------------+
|                page|
+--------------------+
|              Cancel|
|    Submit Downgrade|
|         Thumbs Down|
|                Home|
|           Downgrade|
|         Roll Advert|
|              Logout|
|       Save Settings|
|Cancellation Conf...|
|               About|
| Submit Registration|
|            Settings|
|               Login|
|            Register|
|     Add to Playlist|
|          Add Friend|
|            NextSong|
|           Thumbs Up|
|                Help|
|             Upgrade|
|               Error|
|      Submit Upgrade|
+--------------------+

In [6]:
df.select('auth').distinct().show(50)

VBox()

+----------+
|      auth|
+----------+
|Logged Out|
| Cancelled|
|     Guest|
| Logged In|
+----------+

## Getting rid of unnecessary columns

In [7]:
df_clean = df.select('artist','auth','firstName','gender','lastName','length','level','location','page','song','ts','userId')

VBox()

In [8]:
df_clean.head(10)

VBox()

[Row(artist=u'Popol Vuh', auth=u'Logged In', firstName=u'Shlok', gender=u'M', lastName=u'Johnson', length=524.32934, level=u'paid', location=u'Dallas-Fort Worth-Arlington, TX', page=u'NextSong', song=u'Ich mache einen Spiegel - Dream Part 4', ts=1538352001000, userId=u'1749042'), Row(artist=u'Los Bunkers', auth=u'Logged In', firstName=u'Vianney', gender=u'F', lastName=u'Miller', length=238.39302, level=u'paid', location=u'San Francisco-Oakland-Hayward, CA', page=u'NextSong', song=u'Mi\xc3\x83\xc2\xa9ntele', ts=1538352002000, userId=u'1563081'), Row(artist=u'Lush', auth=u'Logged In', firstName=u'Vina', gender=u'F', lastName=u'Bailey', length=140.35546, level=u'paid', location=u'Hilo, HI', page=u'NextSong', song=u'Baby Talk', ts=1538352002000, userId=u'1697168'), Row(artist=u'Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', auth=u'Logged In', firstName=u'Andres', gender=u'M', lastName=u'Foley', length=277.15873, level=u'paid', location=u'Watertown, SD', page=u'Nex

In [9]:
df_churn = df_clean.groupby('userId').agg(collect_list('auth').alias("auths"))

VBox()

In [10]:
churned = udf(lambda x: 'Cancelled' in x)

VBox()

In [11]:
df_churn = df_churn.withColumn("Churned", churned(df_churn.auths))
df_churn = df_churn.drop('auths')

VBox()

In [12]:
df_label = df_churn.join(df_clean,'userId')

VBox()

In [13]:
df_label.show()

VBox()

+-------+-------+--------------------+---------+---------+------+--------+---------+-----+-----------+-------------+--------------------+-------------+
| userId|Churned|              artist|     auth|firstName|gender|lastName|   length|level|   location|         page|                song|           ts|
+-------+-------+--------------------+---------+---------+------+--------+---------+-----+-----------+-------------+--------------------+-------------+
|1000280|   true|            Coldplay|Logged In|    Frank|     M|  Warren|307.51302| free|Findlay, OH|     NextSong|              Clocks|1538428238000|
|1000280|   true|  Charttraxx Karaoke|Logged In|    Frank|     M|  Warren|225.17506| free|Findlay, OH|     NextSong|           Fireflies|1538428545000|
|1000280|   true|        Dixie Chicks|Logged In|    Frank|     M|  Warren|351.84281| free|Findlay, OH|     NextSong|             Lullaby|1538428770000|
|1000280|   true|             Erasure|Logged In|    Frank|     M|  Warren|242.28526| fre

In [14]:
df_label.persist(StorageLevel.MEMORY_AND_DISK)

VBox()

DataFrame[userId: string, Churned: string, artist: string, auth: string, firstName: string, gender: string, lastName: string, length: double, level: string, location: string, page: string, song: string, ts: bigint]

In [15]:
df_label.select(["userId","Churned"]).distinct().groupBy("Churned").count().collect()

VBox()

[Row(Churned=u'false', count=17275), Row(Churned=u'true', count=5003)]

### Check for nulls

In [16]:
df_label.select([count(when(isnull(c), c)).alias(c) for c in df_label.columns]).show()

VBox()

+------+-------+-------+----+---------+------+--------+-------+-----+--------+----+-------+---+
|userId|Churned| artist|auth|firstName|gender|lastName| length|level|location|page|   song| ts|
+------+-------+-------+----+---------+------+--------+-------+-----+--------+----+-------+---+
|     0|      0|5408927|   0|   778479|778479|  778479|5408927|    0|  778479|   0|5408927|  0|
+------+-------+-------+----+---------+------+--------+-------+-----+--------+----+-------+---+

### Add Features

In [17]:
thumbsUp = df_label.where(df_label.page=='Thumbs Up').groupby("userId").agg(count(col('page')).alias('ThumbsUp')).orderBy('userId')

VBox()

In [18]:
thumbsDown = df_label.where(df_label.page=='Thumbs Down').groupby("userId").agg(count(col('page')).alias('ThumbsDown')).orderBy('userId')

VBox()

In [19]:
allThumbs = thumbsUp.join(thumbsDown,'userId')

VBox()

In [20]:
songsPlayed = df_label.where(col('song')!='null').groupby("userId").agg(count(col('song')).alias('SongsPlayed')).orderBy('userId')

VBox()

In [21]:
df_features = df_churn.join(songsPlayed,'userId')
df_features = df_features.join(allThumbs,'userId')

VBox()

In [22]:
days = df_label.groupby('userId').agg(((max(col('ts')) - min(col('ts')))/86400000).alias("Days"))
df_features = df_features.join(days, "userId")

VBox()

In [23]:
df_features.select([count(when(isnull(c), c)).alias(c) for c in ["userId", "SongsPlayed", "ThumbsUp", "ThumbsDown", "Days"]]).show()

VBox()

+------+-----------+--------+----------+----+
|userId|SongsPlayed|ThumbsUp|ThumbsDown|Days|
+------+-----------+--------+----------+----+
|     0|          0|       0|         0|   0|
+------+-----------+--------+----------+----+

In [24]:
upPerSong = udf(lambda numUp, songs: float(numUp)/float(songs), FloatType())
downPerSong = udf(lambda numDown, songs: float(numDown)/float(songs), FloatType())
songsPerHour = udf(lambda numSongs, numDays: float(numSongs)/float((numDays*24)), FloatType())

VBox()

In [25]:
columns = ['id', 'num1', 'num2']
vals = [
     (1, 8, 4),
     (2, 7, 5)
]
smalldf = spark.createDataFrame(vals, columns)

VBox()

In [26]:
smalldf.drop("UpPerSong")
smalldf = smalldf.withColumn("UpPerSong", upPerSong(smalldf.num1, smalldf.num2))
smalldf.show()

VBox()

+---+----+----+---------+
| id|num1|num2|UpPerSong|
+---+----+----+---------+
|  1|   8|   4|      2.0|
|  2|   7|   5|      1.4|
+---+----+----+---------+

In [27]:
df_features = df_features.withColumn("UpPerSong", upPerSong(df_features.ThumbsUp, df_features.SongsPlayed))

VBox()

In [28]:
df_features.show()

VBox()

+-------+-------+-----------+--------+----------+------------------+-----------+
| userId|Churned|SongsPlayed|ThumbsUp|ThumbsDown|              Days|  UpPerSong|
+-------+-------+-----------+--------+----------+------------------+-----------+
|1000280|   true|       1022|      53|        33|43.075775462962966|  0.0518591|
|1002185|  false|       1778|      92|        14| 57.97252314814815|0.051743533|
|1017805|  false|        250|       7|         4|24.672766203703702|      0.028|
|1030587|  false|       1472|      66|        16| 49.20280092592593|0.044836957|
|1033297|  false|        236|      10|         3| 33.12201388888889|0.042372882|
|1057724|  false|       3847|     200|        29|         58.329375| 0.05198856|
|1059049|  false|        559|      29|         6| 37.39221064814815|0.051878355|
|1069552|  false|        455|      26|         6|55.920520833333335|0.057142857|
|1071308|   true|       1409|      74|        12| 38.46319444444445| 0.05251952|
|1076191|   true|         47

In [29]:
df_features = df_features.withColumn("DownPerSong", downPerSong(df_features.ThumbsDown, df_features.SongsPlayed))

VBox()

In [30]:
df_features = df_features.withColumn("SongsPerHour", songsPerHour(df_features.SongsPlayed, df_features.Days))

VBox()

In [31]:
df_features.select("SongsPerHour", "Churned").groupby("Churned").agg(avg(col('SongsPerHour'))).show()

VBox()

+-------+-----------------+
|Churned|avg(SongsPerHour)|
+-------+-----------------+
|  false|1.076505910115338|
|   true|2.284715742469593|
+-------+-----------------+

In [32]:
df_features.select([count(when(isnull(c), c)).alias(c) for c in ["SongsPlayed", "UpPerSong", "DownPerSong", "Days", "SongsPerHour"]]).show()

VBox()

+-----------+---------+-----------+----+------------+
|SongsPlayed|UpPerSong|DownPerSong|Days|SongsPerHour|
+-----------+---------+-----------+----+------------+
|          0|        0|          0|   0|           0|
+-----------+---------+-----------+----+------------+

In [33]:
assembler = VectorAssembler(inputCols=["SongsPlayed", "UpPerSong", "DownPerSong", "Days", "SongsPerHour"], outputCol="FeatureVector")
df_features = assembler.transform(df_features)

VBox()

In [34]:
scaler = StandardScaler(inputCol="FeatureVector", outputCol="ScaledFeatures", withStd=True)
scalerModel = scaler.fit(df_features)
df_features = scalerModel.transform(df_features)

VBox()

In [35]:
df_features.show()

VBox()

+-------+-------+-----------+--------+----------+------------------+-----------+------------+------------+--------------------+--------------------+
| userId|Churned|SongsPlayed|ThumbsUp|ThumbsDown|              Days|  UpPerSong| DownPerSong|SongsPerHour|       FeatureVector|      ScaledFeatures|
+-------+-------+-----------+--------+----------+------------------+-----------+------------+------------+--------------------+--------------------+
|1000280|   true|       1022|      53|        33|43.075775462962966|  0.0518591| 0.032289628|    0.988568|[1022.0,0.0518590...|[0.91242495298452...|
|1002185|  false|       1778|      92|        14| 57.97252314814815|0.051743533| 0.007874016|   1.2779043|[1778.0,0.0517435...|[1.58736943875390...|
|1017805|  false|        250|       7|         4|24.672766203703702|      0.028|       0.016|   0.4221929|[250.0,0.02800000...|[0.22319592783378...|
|1030587|  false|       1472|      66|        16| 49.20280092592593|0.044836957| 0.010869565|   1.2465415|

In [36]:
convertToInt = udf(lambda x: 1 if x=="true" else 0, IntegerType())

VBox()

In [37]:
df_features = df_features.withColumn('label', convertToInt(df_features.Churned))

VBox()

### Split data into train, test and validation

In [38]:
df_features.show()

VBox()

+-------+-------+-----------+--------+----------+------------------+-----------+------------+------------+--------------------+--------------------+-----+
| userId|Churned|SongsPlayed|ThumbsUp|ThumbsDown|              Days|  UpPerSong| DownPerSong|SongsPerHour|       FeatureVector|      ScaledFeatures|label|
+-------+-------+-----------+--------+----------+------------------+-----------+------------+------------+--------------------+--------------------+-----+
|1000280|   true|       1022|      53|        33|43.075775462962966|  0.0518591| 0.032289628|    0.988568|[1022.0,0.0518590...|[0.91242495298452...|    1|
|1002185|  false|       1778|      92|        14| 57.97252314814815|0.051743533| 0.007874016|   1.2779043|[1778.0,0.0517435...|[1.58736943875390...|    0|
|1017805|  false|        250|       7|         4|24.672766203703702|      0.028|       0.016|   0.4221929|[250.0,0.02800000...|[0.22319592783378...|    0|
|1030587|  false|       1472|      66|        16| 49.20280092592593|0.

In [39]:
train, test = df_features.randomSplit([0.8, 0.2], seed=42)
train, validation = train.randomSplit([0.8, 0.2], seed=42)

VBox()

### Train the model using default parameters and Logistic Regression

In [40]:
lr = LogisticRegression(featuresCol = 'FeatureVector', labelCol = 'label', maxIter=10)
lrModel = lr.fit(train)

VBox()

In [41]:
trainingSummary = lrModel.summary

VBox()

In [42]:
objectiveHistory = trainingSummary.objectiveHistory
print("objectiveHistory:")
for objective in objectiveHistory:
    print(objective)

# Obtain the receiver-operating characteristic as a dataframe and areaUnderROC.
trainingSummary.roc.show()
print("areaUnderROC: " + str(trainingSummary.areaUnderROC))

VBox()

objectiveHistory:
0.539919604573
0.509239460563
0.475498528905
0.442098634351
0.437150329445
0.424084673606
0.402606328971
0.385793271091
0.373180789382
0.363566860268
0.362864432603
+--------------------+-------------------+
|                 FPR|                TPR|
+--------------------+-------------------+
|                 0.0|                0.0|
|0.001330195436406426|0.01673497267759563|
|0.002251099969303...| 0.0351775956284153|
|0.003888263583341...|0.05430327868852459|
|0.004706845390361199|0.07342896174863388|
|0.006344009004399...|0.08913934426229508|
|0.007264913537296634|0.10792349726775956|
|0.009106722603090147|0.12295081967213115|
|0.010232272587741738|0.14173497267759563|
|0.011869436201780416|0.15915300546448088|
|0.013097308912309424|0.17691256830601093|
|0.013915890719328763|0.19637978142076504|
|0.016166990688631946|0.21174863387978143|
|0.017701831576793205|0.22882513661202186|
|0.019134349739077048|0.24453551912568305|
| 0.02066919062723831| 0.2599043715846995|


In [43]:
fMeasure = trainingSummary.fMeasureByThreshold
maxFMeasure = fMeasure.groupBy().max('F-Measure').select('max(F-Measure)').head()
bestThreshold = fMeasure.where(fMeasure['F-Measure'] == maxFMeasure['max(F-Measure)']) \
    .select('threshold').head()['threshold']

VBox()

In [44]:
fMeasure.show()

VBox()

+------------------+--------------------+
|         threshold|           F-Measure|
+------------------+--------------------+
|0.9710608906938889|0.032775919732441476|
|0.9122627800746941|  0.0674746151326564|
|0.8943524742261496|             0.10176|
|0.8800415670707707|  0.1348385073690812|
|0.8698472603888472|  0.1605659796985543|
|0.8586304962166017| 0.19064856711915537|
|0.8497349234519836| 0.21320698845128813|
|0.8401395549021052| 0.24106883531803663|
| 0.831604479824745|  0.2655270655270655|
|0.8206441633376872|  0.2898712926692781|
| 0.811713169650478|  0.3160208848584776|
|0.8037067685745739|  0.3345925526173772|
|0.7950108817112244|  0.3553434102360117|
|0.7860898987524877|  0.3737927434090316|
|0.7783106379253321|  0.3911590850681059|
|0.7682660595076778| 0.41032127498102705|
|0.7575473779758173|  0.4272908366533864|
|0.7477319327542816| 0.44264705882352945|
|0.7374324443644824| 0.45979232069548415|
|0.7265856661884406|  0.4709800190294957|
+------------------+--------------

In [45]:
print(bestThreshold)
print(maxFMeasure)

VBox()

0.299504098269
Row(max(F-Measure)=0.6614370881706935)

In [46]:
pr = trainingSummary.pr
pr.show()

VBox()

+-------------------+------------------+
|             recall|         precision|
+-------------------+------------------+
|                0.0|0.7903225806451613|
|0.01673497267759563|0.7903225806451613|
| 0.0351775956284153|             0.824|
|0.05430327868852459|0.8071065989847716|
|0.07342896174863388|0.8237547892720306|
|0.08913934426229508|0.8080495356037152|
|0.10792349726775956|0.8165374677002584|
|0.12295081967213115|0.8017817371937639|
|0.14173497267759563|0.8058252427184466|
|0.15915300546448088|0.8006872852233677|
|0.17691256830601093|0.8018575851393189|
|0.19637978142076504|0.8087201125175809|
|0.21174863387978143|0.7969151670951157|
|0.22882513661202186|0.7947805456702254|
|0.24453551912568305|0.7929125138427464|
| 0.2599043715846995|0.7902388369678089|
| 0.2769808743169399| 0.791219512195122|
| 0.2930327868852459|0.7886029411764706|
| 0.3084016393442623|0.7838541666666666|
| 0.3251366120218579|0.7848309975267931|
+-------------------+------------------+
only showing top

### Make predictions on validation set

In [47]:
predictions = lrModel.transform(validation)

VBox()

In [48]:
predictions.show()

VBox()

+-------+-------+-----------+--------+----------+------------------+-----------+------------+------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
| userId|Churned|SongsPlayed|ThumbsUp|ThumbsDown|              Days|  UpPerSong| DownPerSong|SongsPerHour|       FeatureVector|      ScaledFeatures|label|       rawPrediction|         probability|prediction|
+-------+-------+-----------+--------+----------+------------------+-----------+------------+------------+--------------------+--------------------+-----+--------------------+--------------------+----------+
|1033297|  false|        236|      10|         3| 33.12201388888889|0.042372882| 0.012711864|  0.29688212|[236.0,0.04237288...|[0.21069695587509...|    0|[0.84515469207257...|[0.69954974183046...|       0.0|
|1057724|  false|       3847|     200|        29|         58.329375| 0.05198856|0.0075383415|   2.7480435|[3847.0,0.0519885...|[3.43453893750634...|    0|[1.43632507056

In [49]:
print(predictions.filter(predictions.label == predictions.prediction).count())
print(predictions.count())

VBox()

2662
3186

In [50]:
print(2662/3186)*100

VBox()

0

In [51]:
evaluator = BinaryClassificationEvaluator()
print('Area Under ROC', evaluator.evaluate(predictions))

VBox()

('Area Under ROC', 0.8617849953633543)

### Train the model using random forest

In [52]:
#random forest classifier model
rf = RandomForestClassifier(numTrees=10,featuresCol = 'FeatureVector', labelCol = 'label')
rf_model = rf.fit(train)
rf_preds = rf_model.transform(validation)
rf_preds1 = rf_model.transform(test)

VBox()

In [53]:
predictions = rf_preds
print(predictions.filter(predictions.label == predictions.prediction).count())
print(predictions.count())

VBox()

2768
3186

In [54]:
evaluator = BinaryClassificationEvaluator()
print('Area Under ROC', evaluator.evaluate(predictions))

VBox()

('Area Under ROC', 0.8862554565399315)

In [55]:
predictions = rf_preds1
print(predictions.filter(predictions.label == predictions.prediction).count())
print(predictions.count())

VBox()

3498
4005

In [56]:
evaluator = BinaryClassificationEvaluator()
print('Area Under ROC', evaluator.evaluate(predictions))

VBox()

('Area Under ROC', 0.899103510074916)

### Gradient Boosted Tress

In [57]:
#gradient boosted trees (ie ada boost)
gbtrees = GBTClassifier(maxIter=10,featuresCol = 'FeatureVector', labelCol = 'label')
gbtree_model = gbtrees.fit(train)
gbtree_preds = gbtree_model.transform(validation)
gbtree_preds1 = gbtree_model.transform(validation)

VBox()

Exception in thread cell_monitor-57:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.6/site-packages/awseditorssparkmonitoringwidget-1.0-py3.6.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 1863



In [58]:
predictions = gbtree_preds
print(predictions.filter(predictions.label == predictions.prediction).count())
print(predictions.count())

VBox()

2757
3186

In [59]:
evaluator = BinaryClassificationEvaluator()
print('Area Under ROC', evaluator.evaluate(predictions))

VBox()

('Area Under ROC', 0.8928609798023202)

In [60]:
predictions = gbtree_preds1
print(predictions.filter(predictions.label == predictions.prediction).count())
print(predictions.count())

VBox()

2757
3186

In [61]:
evaluator = BinaryClassificationEvaluator()
print('Area Under ROC', evaluator.evaluate(predictions))

VBox()

('Area Under ROC', 0.8928609798023205)